<a href="https://colab.research.google.com/github/xszbencex/SZFM_2021_10_GitGud_Big/blob/main/SZFM_2021_10_GitGud_Big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
### **COVID19 prediction and forecasting using machine learning**
---